<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/notebooks/modelling/kk_mvp_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
===================================================
Author: Kasia Kirby
Role: Reporting Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/kasia-kirby
Date: 2025-02-17
Version: 1.0

Description:
    This notebook builds the end to end NLP pipeline to run topic modelling,
    sentiment analysis, and summarisation on JPM quarterly call trancripts from 2023-2024.

===================================================
"""

'\n===================================================\nAuthor: Kasia Kirby\nRole: Reporting Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/kasia-kirby\nDate: 2025-02-17\nVersion: 1.0\n\nDescription:\n    This notebook builds the end to end NLP pipeline to run topic modelling, \n    sentiment analysis, and summarisation on JPM quarterly call trancripts from 2023-2024.  \n\n===================================================\n'

**Task**

Model the transcript data by analyst and by selected quarters using BERTopic, FinBert and LLM model (Flan-T5).

**Data**

JMP bank, 2023-2024, Management & Q&A transcripts

**Requirements**

1. finBERt returns sentiment accuracy
2. LLM model returns better sentiment accuracy
3. BERTopic performs accurate topic extraction
4. LLM is able to capture more topic context and provide summarisation of those
5. Analyst topics of interest and sentiment varies widely quarter by quarter
6. Topics and sentiment comparison between analysts.

**Approach**

Topic model -> Sentiment analysis -> Insight + Comparison

- First, topics are identified freely by the model on each transcript
- Second the model is trained with G-SIB assessment topics
- Third, sentiment analysis is run on the resulting four files (Q&A and management - with free topics, and trained topics)
- Fourth, all are compared against each other to find best insights

! Test the pipeline on a single analyst and one quarter (Q4 2024) before scaling up.

**Benchmark analysis**

- **comparison between model types and their results** (apply like-for-like rules when comparing the models results i.e. if we gather insights on a specific analyst in a specific quarter (Q42024) compare models results and ability to capture correct information using the same analyst and quarter across all models),
- **comparison between Q42024 analyst insights and other quarters insights** (apply full models flow across at least two different periods i.e. Q4 2024 and Q2 2024 and choose a specific analyst and check what major topics and sentiment they cover between the two periods; do they have similar sentiment over time? Do they focus on specific topic areas that are of interest in that specific quarter over the other quarter?),
- **comparison between two different analysts in the same quarter** (are analysts interested in different topics? What can we extract at analyst level with regards to sentiment, topics of interest and conversation summarisation?)



# 1. Import libraries and files

In [3]:
import os
import sys
from google.colab import drive

import pandas as pd
import matplotlib.pyplot as plt

In [5]:
# Mount Google Drive to the root location
drive.mount('/content/drive', force_remount=True)
BOE_path = '/content/drive/MyDrive/BOE/bank_of_england/data/preprocessed_data/JP Mogran processed thru OpenAI'
print(os.listdir(BOE_path))

Mounted at /content/drive
['4Q24_QNA.csv', '3Q24_QNA.csv', '2Q24_QNA.csv', '1Q24_QNA.csv', '4Q23_QNA.csv', '3Q23_QNA.csv', '2Q23_QNA.csv', '1Q23_QNA.csv', 'JPMorgan_QNA_Consildated_processed_data.xlsx', 'MD_4Q24.csv', 'MD_3Q24.csv', 'MD_2Q24.csv', 'MD_1Q24.csv', 'MD_4Q23.csv', 'MD_3Q23.csv', 'MD_2Q23.csv', 'MD_1Q23.csv', 'JPMorgan_Management_Discussion_processed_data.xlsx']


In [6]:
qna_path = os.path.join(BOE_path, 'JPMorgan_QNA_Consildated_processed_data.xlsx')
management_path = os.path.join(BOE_path, 'JPMorgan_Management_Discussion_processed_data.xlsx')

df_qna = pd.read_excel(qna_path)
df_management = pd.read_excel(management_path)


print("First few rows of Q&A transcript data:")
display(df_qna.head())

print("First few rows of Management Discussion transcript data:")
display(df_management.head())

print("\nQ&A Data Info:")
df_qna.info()

print("\nManagement Discussion Data Info:")
df_management.info()

First few rows of Q&A transcript data:


,Index,Quarter-Year,Question,Asked By,Role of the person asked the question,Answer,Answered By,Role of the person answered the question
0,1,1Q23,"So, Jamie, I was actually hoping to get your p...",Steven Chubak,"Analyst, Wolfe Research LLC","Well, I think you were already kind of complet...",Jamie Dimon,"Chairman & Chief Executive Officer, JPMorgan C..."
1,2,1Q23,"Hey, thanks. Good morning. Hey, Jeremy, I was ...",Ken Usdin,"Analyst, Jefferies LLC","Yeah, sure. So let me just summarize the drive...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co."
2,3,1Q23,"Hi, thanks. Jeremy, wanted to follow up again ...",John McDonald,"Analyst, Autonomous Research","Yeah. John, it's a really good question, and w...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co."
3,4,1Q23,My first question is you mentioned that your r...,Erika Najarian,"Analyst, UBS Securities LLC","Yeah. So, Erika, as you know, we take \n not g...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co."
4,5,1Q23,Hey. Good morning. Maybe just a little bit on ...,Jim Mitchell,"Analyst, Seaport Global Securities LLC","Yeah. A couple things there. So, first of all,...","Jeremy Barnum, Jamie Dimon","Chief Financial Officer, JPMorgan Chase & Co.;..."


First few rows of Management Discussion transcript data:


,Index,Quarter-Year,Text
0,NaN,4Q24,MANAGEMENT DISCUSSION SECTION \n \nOperator : ...
1,NaN,3Q24,MANAGEMENT DISCUSSION SECTION \n \n...
2,NaN,2Q24,MANAGEMENT DISCUSSION SECTION \n \n...
3,NaN,1Q24,MANAGEMENT DISCUSSION SECTION \n \n...
4,NaN,4Q23,MANAGEMENT DISCUSSION SECTION \n \n...



Q&A Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 8 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   Index                                     91 non-null     int64 
 1   Quarter-Year                              91 non-null     object
 2   Question                                  90 non-null     object
 3   Asked By                                  90 non-null     object
 4   Role of the person asked the question     90 non-null     object
 5   Answer                                    89 non-null     object
 6   Answered By                               89 non-null     object
 7   Role of the person answered the question  89 non-null     object
dtypes: int64(1), object(7)
memory usage: 5.8+ KB

Management Discussion Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns)

# 2. Data preparation

Load and preprocess transcripts (ensure tokenization, text cleaning, etc.).
Structure the data by analyst and quarter.

# 3. Topic Modeling (BERTopic)

Run unsupervised BERTopic on Management & Q&A transcripts.
Train BERTopic on G-SIB assessment topics for a supervised version.

# 4. Sentiment Analysis (finBERT, Flan-T5)

Apply FinBERT to extract sentiment scores.
Apply Flan-T5 (or another LLM) to re-evaluate sentiment and capture better sentiment accuracy.

# 5. Summarisation & Context Extraction

Use Flan-T5 to summarize extracted topics.
Compare free topics vs G-SIB-trained topics.

# 6. Comparative Analysis

1. Model Comparisons -
Compare BERTopic, FinBERT, and LLM results for accuracy.
Ensure like-for-like comparison across different model outputs.
2. Quarter-on-Quarter Analyst Comparison -
Select at least two quarters (e.g., Q4 2024 vs. Q2 2024).
Compare topics and sentiment shifts for a chosen analyst.
3. Analyst Comparisons (Same Quarter) -
Identify topic differences between analysts within the same quarter.
Check sentiment variation across analysts.1.